# Distracted Driver Notebook w/ Precomputed Conv Features
This notebook attempts to achieve much the same thing as the other distracted-driver notebook, but it uses precomputed convolutional features from the fixed layers and runs forward computation on only the fine-tuned layers

In [7]:
%matplotlib inline
import os
import math
import keras
import keras.preprocessing.image as image
import numpy as np

## Load and Modify Model
First the full VGG16 model is retrieved.

In [8]:
model = keras.applications.VGG16()

Then we remove the affine layers

In [9]:
# Remove affine layers.
model.layers.pop()
model.layers.pop()
model.layers.pop()

# Remove flatten layer and max pool
model.layers.pop()
model.layers.pop()

# Passing these layers as arguments to a new sequential model removes the dense outputs (shape=(N, 1000)) and
# changes them to shape=(N, 14, 14, 512).
model = keras.models.Sequential(model.layers)

Print layers to confirm everything went as planned. Last layer should be conv or max pool.

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

## Load Training Data

In [11]:
training_dir = '../../data/distracted_drivers/sample/train'
validation_dir = '../../data/distracted_drivers/sample/valid/'

# training data
train_gen = image.ImageDataGenerator()
training_batches = train_gen.flow_from_directory(training_dir, target_size=(224, 224), shuffle=False)
trn_classes = training_batches.classes
train_features = model.predict_generator(
    training_batches,
    math.ceil(training_batches.n / training_batches.batch_size)
)

# validation data
val_gen = image.ImageDataGenerator()
validation_batches = val_gen.flow_from_directory(validation_dir, target_size=(224, 224), shuffle=False)
val_classes = validation_batches.classes
validation_features = model.predict_generator(
    validation_batches,
    math.ceil(validation_batches.n / validation_batches.batch_size)
)

Found 50 images belonging to 10 classes.
Found 50 images belonging to 10 classes.


Save the features and classes

In [12]:
np.save("training_features", train_features)
np.save("validation_features", validation_features)
np.save("training_classes", trn_classes)
np.save("validation_classes", val_classes)

## Train FC Model on Convolutional Features

Build final layers

In [22]:
model_fc = keras.models.Sequential([
    keras.layers.MaxPooling2D(input_shape=model.layers[-1].output_shape[1:]),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.0),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.0),
    keras.layers.Dense(10, activation='softmax')
])

opt = keras.optimizers.RMSprop(lr=0.00001, rho=0.7)

model_fc.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

Convert classes to one-hot encoding

In [24]:
trn_classes_cat = keras.utils.np_utils.to_categorical(trn_classes)
val_classes_cat = keras.utils.np_utils.to_categorical(val_classes)

Train model

In [27]:
model_fc.fit(train_features, trn_classes_cat, epochs=100,
             batch_size=10, validation_data=(validation_features, val_classes_cat))

Train on 50 samples, validate on 50 samples
Epoch 1/100
50/50 [==============================] - 5s - loss: 2.5789 - acc: 0.8400 - val_loss: 10.5870 - val_acc: 0.2000
Epoch 2/100
50/50 [==============================] - 5s - loss: 2.5789 - acc: 0.8400 - val_loss: 10.5856 - val_acc: 0.2000
Epoch 3/100
50/50 [==============================] - 5s - loss: 2.5789 - acc: 0.8400 - val_loss: 10.5837 - val_acc: 0.2000
Epoch 4/100
50/50 [==============================] - 5s - loss: 2.5789 - acc: 0.8400 - val_loss: 10.5822 - val_acc: 0.2000
Epoch 5/100
50/50 [==============================] - 5s - loss: 2.5789 - acc: 0.8400 - val_loss: 10.5761 - val_acc: 0.2000
Epoch 6/100
50/50 [==============================] - 5s - loss: 2.5789 - acc: 0.8400 - val_loss: 10.5787 - val_acc: 0.2000
Epoch 7/100
50/50 [==============================] - 5s - loss: 2.5789 - acc: 0.8400 - val_loss: 10.5787 - val_acc: 0.2000
Epoch 8/100
50/50 [==============================] - 5s - loss: 2.5789 - acc: 0.8400 - val_loss

KeyboardInterrupt: 

In [21]:
model_fc.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
max_pooling2d_6 (MaxPooling2 (None, 7, 7, 512)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 4096)              102764544 
_________________________________________________________________
dropout_3 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dropout_4 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 4096)              16781312  
Total para